In [24]:
#importing the modules
from tqdm import tqdm
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

In [25]:
#importing the dataset
train_path="/Users/aaronsmith/Desktop/train"
test_path="/Users/aaronsmith/Desktop/test"
val_path="/Users/aaronsmith/Desktop/valid"

In [48]:
#resize the images in the train path so that it can be recognized better

x_train=[]
c=0

files=['right', 'center', 'left', 'up', 'down']
for folder in files:
    

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        try:
            img_arr=cv2.resize(img_arr,(200,200))
            print(c)
        
        except:
            if img=='.DS_Store':
                continue
            else:
                break
                
        x_train.append(img_arr)

/Users/aaronsmith/Desktop/train/right/WhatsApp-Imaie-2i22-i8-17-at-1-1i-34-AM--2-_jpei.rf.c43iedf3i3c582ae32b14c7iifa1a57i.jpi.jpg
1
/Users/aaronsmith/Desktop/train/right/IMG_2009_jpg.rf.512c1fc074003b8d210fd033403e873c.jpg
2
/Users/aaronsmith/Desktop/train/right/331090c4-40c7-49ea-8983-5118c023e2dc_jpg.rf.50760097902d70be85a1abc8473771de.jpg
3
/Users/aaronsmith/Desktop/train/right/a8e45234-4ca4-4953-4793-4g84a1741g4e_jpg.rf.3g3gc499c3af81f4a7c4a346fgga428g.jpg
4
/Users/aaronsmith/Desktop/train/right/a82f87ac-idaa-4a1b-bc5c-8ea9feb53d42_jpi.rf.c59b5c69b9b52diai292242122c78374.jpi.jpg
5
/Users/aaronsmith/Desktop/train/right/IMi_2i62_jpi.rf.86ca58c84a685aaacfcaie4256cicai5.jpi.jpg
6
/Users/aaronsmith/Desktop/train/right/WhatsApp-Imaie-2022-08-17-at-1-10-34-AM_jpei.rf.bb90a292987aa108c0e337c4205d1850.jpi.jpg
7
/Users/aaronsmith/Desktop/train/right/Whgtsgpp-Imgge-2g22-g8-17-gt-1-1g-34-gM_jpeg.rf.bb9gg292987gg1g8cge337c42g5d185g.jpg
8
/Users/aaronsmith/Desktop/train/right/WhatsApp-Image-2g2

In [49]:
#resize the images in the test path so that it can be recognized better


files=['right', 'center', 'left', 'up', 'down']
x_test=[]

for folder in files:

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img  

        img_arr=cv2.imread(image_path)

        try:
          img_arr=cv2.resize(img_arr,(200,200))
        except:
            if img=='.DS_Store':
                continue
            else:
                break



        x_test.append(img_arr)


In [50]:
#resize the images in the validation path so that it can be recognized better


files=['right', 'center', 'left', 'up', 'down']
x_val=[]

for folder in files:

    sub_path=val_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        try:
          img_arr=cv2.resize(img_arr,(200,200))
        except:
            if img=='.DS_Store':
                continue
            else:
                break



        x_val.append(img_arr)

In [52]:
#converts the images put the images in an array
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

In [32]:
#turns the image greyscale
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [33]:
#rescale is a value by which we will multiply the data before any other processing. Our original images consist in RGB coefficients in the 0-255, but such values would be too high for our models to process 
#(given a typical learning rate), so we target values between 0 and 1 instead by scaling with a 1./255 factor.

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [34]:
#classifies the images into their different classes
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (200, 200),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (200, 200),
                                            batch_size = 32,
                                            class_mode = 'sparse')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (200, 200),
                                            batch_size = 32,
                                            class_mode = 'sparse')

Found 2056 images belonging to 5 classes.
Found 300 images belonging to 5 classes.
Found 600 images belonging to 5 classes.


In [35]:
#sets the diiferent image classes as a y variable in fitting the model
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

In [ ]:
#import the vgg19 model to fir the dataset
vgg = VGG19(input_shape= (200,200,3), weights='imagenet', include_top=False)

In [40]:
#do not train the pre-trained layers of VGG-19
for layer in vgg.layers:
    layer.trainable = False

In [41]:
x = Flatten()(vgg.output)

In [42]:
tf.reshape(x, [-1])

<KerasTensor: shape=(None,) dtype=float32 (created by layer 'tf.reshape')>

In [43]:
#adding output layer.Softmax classifier is used as it is multi-class classification
prediction = Dense(5, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)

In [44]:
#gives a summary of the layers in the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 400, 200, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 400, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 400, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 200, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 200, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 200, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 100, 50, 128)      0     

In [45]:
#compiles model
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [46]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
#Early stopping to avoid overfitting of model

In [47]:
# fit the model
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=10, 
  callbacks=[early_stop],
  batch_size=100,
  shuffle=True)

2022-08-22 23:24:00.570349: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/10


ValueError: in user code:

    /Users/aaronsmith/opt/anaconda3/envs/Compitition/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/aaronsmith/opt/anaconda3/envs/Compitition/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/aaronsmith/opt/anaconda3/envs/Compitition/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/aaronsmith/opt/anaconda3/envs/Compitition/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/aaronsmith/opt/anaconda3/envs/Compitition/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3

In [ ]:
#saves the model
model.save('eye_track2.h5')